In [ ]:
import requests
from PIL import Image
import io
import time
import pandas as pd
import re
from bs4 import BeautifulSoup
import h5py
import numpy as np

## Getting healy cruises 

In [ ]:
r = requests.get("http://icefloe.net/Aloftcon_Photos/albums/2015/20150809-2201.jpeg")

In [ ]:
im = Image.open(io.BytesIO(r.content))
im

In [ ]:
for x in range(0, 24):
    x = str(x)
    count = "20110720-{}01".format("0"+x if len(x) < 2 else x)
    
    try:
        with open('data/'+count+'.jpg', 'xb') as f:
            img = requests.get("http://icefloe.net/Aloftcon_Photos/albums/2011/%s.jpeg" % count)
            
            for chunk in img:
                f.write(chunk)
                
        print("wrote", count, "to disk")
        time.sleep(1)
    except FileExistsError:
        print("file", count, "already exists")

In [ ]:
locations = pd.read_table("HLY1101_1min.r2rnav", header=None, names=["time", "longitude", "latitude", "speed", "heading"], index_col=0, skiprows=3)

In [ ]:
comp_re = re.compile("2011\-07\-20T\d{2}\:(01\:00|00\:59)")
locations[locations.index.str.contains(comp_re)]

In [ ]:
cruises = requests.get("http://www.rvdata.us/catalog/Healy")
cruises = BeautifulSoup(cruises.text, features='lxml')

In [ ]:
cruises.select("a[href*=catalog/]")[6]

In [ ]:
base_url = "http://get.rvdata.us/cruise/{0}/products/r2rnav/{0}_1min.r2rnav"
comp_re = re.compile("T\d{2}\:(01\:0[0-9]|00\:5[0-9])")
locations = None
i = 0

for link in cruises.select("a[href*=catalog/]"):
    
    # check if after 2009
    if int(link.text[3:5]) >= 10:
        print(base_url.format(link.text))
        locations_tmp = pd.read_table(base_url.format(link.text), header=None, 
                                      names=["time", "longitude", "latitude", "speed", "heading", "image"], 
                                      index_col=0, skiprows=3)
        
        if not locations_tmp.empty:
            print(len(locations_tmp))
            if locations is not None:
                locations = locations.append(locations_tmp[locations_tmp.index.str.contains(comp_re)])
            else:
                locations = locations_tmp[locations_tmp.index.str.contains(comp_re)].copy()
        
        
    
    #if i == 0:
    #    break
        
        
    i += 1
    
    
locations.index = pd.to_datetime(locations.index)
locations["ice"] = 254 

In [ ]:
# direct download
base_url_img = "http://icefloe.net/Aloftcon_Photos/albums/{}/{}.jpeg"

for point in locations.index:

    count = point.strftime("%Y%m%d-%H01")

    img = requests.get(base_url_img.format(point.year, count))
    #print("http://icefloe.net/Aloftcon_Photos/albums/{}/{}.jpeg".format(point.year, count))

    locations.loc[point, "image"] = img.content
    time.sleep(1)
    #break

In [ ]:
# prepare list of urls for download, add paths to DataFrame
base_url_img = "http://icefloe.net/Aloftcon_Photos/albums/{}/{}.jpeg"
base_path = "data/images/remote/{}.jpeg"

with open("data/url_list_img", "w") as url_list:
    for point in locations.index:

        count = point.strftime("%Y%m%d-%H01")
        
        url_list.write(base_url_img.format(point.year, count)+ "\n")
        
locations["image"] = locations.index.map(lambda x: base_path.format(x.strftime("%Y%m%d-%H01")))

In [ ]:
locations[((locations['ice'] > 0) & (locations['ice'] <= 100)) | (locations['ice'] == 255) | (locations['ice'] == 252)]

In [ ]:
locations.to_hdf("data/backup/complete_paths.h5", "data")

In [ ]:
locations = pd.read_hdf("data/backup/complete_paths.h5", "data")

In [ ]:
locations

## Ice extent with Lon/Lat 

In [ ]:
tmp = h5py.File("data/extent/remote/NISE_SSMISF17_20150908.h5", 'r')

In [ ]:
extent = tmp["Northern Hemisphere/Data Fields/Extent"]

In [ ]:
def haversine_np(lons, lats, lon2, lat2):
    lons, lats, lon2, lat2 = map(np.radians, [lons, lats, lon2, lat2])

    dlon = lon2 - lons
    dlat = lat2 - lats

    a = np.sin(dlat/2.0)**2 + np.cos(lats) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    
    idx = np.unravel_index(np.argmin(km), lats.shape)
    return idx

In [ ]:
lats = np.fromfile('data/extent/grid/EASE2_N25km.lats.720x720x1.double', dtype=np.float64).reshape((720,720))
lons = np.fromfile('data/extent/grid/EASE2_N25km.lons.720x720x1.double', dtype=np.float64).reshape((720,720))

In [ ]:
locations["ice"] = locations.apply(lambda x: extent[haversine_np(lons, lats, x["longitude"], x["latitude"])], axis=1)

In [ ]:
dates = np.unique(locations.index.date)
base_url_extent = "ftp://n5eil01u.ecs.nsidc.org/SAN/OTHR/NISE.004/{}/NISE_SSMISF17_{}.HDFEOS"
# ftp://n5eil01u.ecs.nsidc.org/SAN/OTHR/NISE.004/2015.08.09/NISE_SSMISF17_20150809.HDFEOS

with open("data/download/url_list_extent", "w") as url_list:
    for date in dates:
        folder = date.strftime("%Y.%m.%d")
        file = date.strftime("%Y%m%d")
        url_list.write(base_url_extent.format(folder, file)+"\n")
    
    #locations.loc[locations.index.date == date, "ice"] = locations[locations.index.date == date].apply(lambda x: extent[haversine_np(lons, lats, x["longitude"], x["latitude"])], axis=1)

In [ ]:
# missing values for 2012-02-01, nise_ssmisf17_20120201.h5
locations = locations.drop(locations.index[locations.index.date == dates[364]])
dates = np.unique(locations.index.date)

for date in dates:
    file = date.strftime("%Y%m%d")
    
    extent = h5py.File("data/extent/remote/NISE_SSMISF17_{}.h5".format(file), 'r')
    extent = extent["Northern Hemisphere/Data Fields/Extent"]
    
    locations.loc[locations.index.date == date, "ice"] = locations[locations.index.date == date].apply(lambda x: extent[haversine_np(lons, lats, x["longitude"], x["latitude"])], axis=1)

In [ ]:
points = np.dstack([lats.ravel(),lons.ravel()])[0].reshape(720,720,2)[0]

In [ ]:
a[haversine_np(lons, lats, -166.477043, 53.924185)]